# 6.1 수정주가 크롤링

## 6.1.1 개별종목 주가 크롤링

- 티커 불러옴

In [4]:
import pandas as pd

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)

In [ ]:
KOR_ticker['종목코드'][0]

- 앞에 0이 삭제됨
- 6자리로 만들어 줌

In [5]:
import numpy as np

KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

In [ ]:
KOR_ticker

In [ ]:
import os
if not os.path.exists('data/KOR_price'):
    os.makedirs('data/KOR_price')

In [ ]:
i = 0
name = KOR_ticker['종목코드'][i]

- 먼저 빈 데이터프레임 만들어줌

In [ ]:
import numpy as np
from datetime import date

price = pd.DataFrame({'Price' : [np.nan]})
price.index = [date.today().strftime("%Y-%m-%d")]

In [ ]:
price

- 삼성전자 주가 예제
- url: https://fchart.stock.naver.com/siseJson.nhn?symbol=005930&requestType=1&startTime=20191117&endTime=20210124&timeframe=day

In [ ]:
from dateutil.relativedelta import relativedelta
import requests as rq

fr = (date.today() + relativedelta(years=-3)).strftime("%Y-%m-%d").replace("-", "")
to = (date.today()).strftime("%Y-%m-%d").replace("-", "")

In [ ]:
url = 'https://fchart.stock.naver.com/siseJson.nhn?symbol='+name+'&requestType=1&startTime='+fr+'&endTime='+to+'&timeframe=day'

In [ ]:
data = rq.get(url).content

head(data)

In [ ]:
from io import BytesIO

data_price = pd.read_csv(BytesIO(data))

In [ ]:
data_price

In [ ]:
import re
from datetime import datetime

price = data_price.iloc[:, [0, 4]]
price.columns = ['Date', 'Price']
price = price.dropna()
price['Date'] = price['Date'].str.extract('(\d+)')
price['Date'] = price['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

price.set_index('Date', inplace=True)

In [ ]:
price

In [ ]:
price.to_csv('data/KOR_price/'+name+'_price.csv')

## 6.1.2 전 종목 주가 크롤링

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import os
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
import re
import time

if not os.path.exists('data/KOR_price'):
    os.makedirs('data/KOR_price')

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)
error_ticker = []

for i in range(0, len(KOR_ticker)): 

    # 빈 데이터프레임 생성
    price = pd.DataFrame({'Price' : [np.nan]})
    price.index = [date.today().strftime("%Y-%m-%d")]
    
    # 티커 선택
    name = KOR_ticker['종목코드'][i]
    
    # 시작일과 종료일 
    fr = (date.today() + relativedelta(years=-3)).strftime("%Y-%m-%d").replace("-", "")
    to = (date.today()).strftime("%Y-%m-%d").replace("-", "")
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        # url 생성
        url = 'https://fchart.stock.naver.com/siseJson.nhn?symbol='+name+'&requestType=1&startTime='+fr+'&endTime='+to+'&timeframe=day'
        
        # 데이터 다운로드
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data)) 
        
        # 필요한 열만 선택 후 클렌징
        price = data_price.iloc[:, [0, 4]]
        price.columns = ['Date', 'Price']
        price = price.dropna()
        price['Date'] = price['Date'].str.extract('(\d+)')
        price['Date'] = price['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
        price.set_index('Date', inplace=True)
    except:
        # 오류 발생시 해당 종목명을 출력하고 다음 루프로 이동
        print('Error in ticker ' + name)
        error_ticker.append(name)
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장
    price.to_csv('data/KOR_price/'+name+'_price.csv')
    
    # 타임슬립 적용
    time.sleep(2)    

## 6.2 재무제표 및 가치지표 크롤링

### 6.2.1 재무제표 다운로드

In [30]:
import os
if not os.path.exists('data/KOR_fs'):
    os.makedirs('data/KOR_fs')

In [31]:
import pandas as pd

url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930'
data = pd.read_html(url)

In [32]:
[item.head(3) for item in data] 

[  IFRS(연결)    2017/12    2018/12    2019/12    2020/09       전년동기  전년동기(%)
 0      매출액  2395754.0  2437714.0  2304009.0  1752555.0  1705161.0      2.8
 1     매출원가  1292907.0  1323944.0  1472395.0  1066834.0  1086850.0     -1.8
 2    매출총이익  1102847.0  1113770.0   831613.0   685721.0   618311.0     10.9,
   IFRS(연결)   2019/12   2020/03   2020/06   2020/09      전년동기  전년동기(%)
 0      매출액  598848.0  553252.0  529661.0  669642.0  620035.0      8.0
 1     매출원가  385545.0  348067.0  319062.0  399705.0  399939.0     -0.1
 2    매출총이익  213302.0  205185.0  210599.0  269937.0  220096.0     22.6,
               IFRS(연결)    2017/12    2018/12    2019/12    2020/09
 0                   자산  3017521.0  3393572.0  3525645.0  3757887.0
 1   유동자산계산에 참여한 계정 펼치기  1469825.0  1746974.0  1813853.0  2036349.0
 2  비유동자산계산에 참여한 계정 펼치기  1547696.0  1646598.0  1711792.0  1721538.0,
               IFRS(연결)    2019/12    2020/03    2020/06    2020/09
 0                   자산  3525645.0  3574575.0  3579595.0  3757887.0
 

In [33]:
data_IS = data[0]
data_BS = data[2]
data_CF = data[4]

In [34]:
data_IS.columns

Index(['IFRS(연결)', '2017/12', '2018/12', '2019/12', '2020/09', '전년동기',
       '전년동기(%)'],
      dtype='object')

In [35]:
data_IS = data_IS.iloc[:, 0:(len(data_IS.columns)-2)]

In [36]:
data_fs = pd.concat([data_IS, data_BS, data_CF]).reset_index(drop=True)
data_fs.iloc[:, 0] = data_fs.iloc[:, 0].replace({'계산에 참여한 계정 펼치기':''}, regex = True)
data_fs = data_fs.set_index(data_fs.columns[0])
data_fs = data_fs[~data_fs.index.duplicated(keep='first')]
data_fs = data_fs.filter(like = '12', axis = 1)

In [37]:
data_fs.head()

,2017/12,2018/12,2019/12
IFRS(연결),,,
매출액,2395754.0,2437714.0,2304009.0
매출원가,1292907.0,1323944.0,1472395.0
매출총이익,1102847.0,1113770.0,831613.0
판매비와관리비,566397.0,524903.0,553928.0
영업이익,536450.0,588867.0,277685.0


In [38]:
data_fs.dtypes

2017/12    float64
2018/12    float64
2019/12    float64
dtype: object

In [39]:
data_fs.to_csv('data/KOR_fs/005930_fs.csv')

## 6.2.2 가치지표 계산하기

In [40]:
if not os.path.exists('data/KOR_value'):
    os.makedirs('data/KOR_value')

In [41]:
value_type = ['지배주주순이익', '자본', '영업활동으로인한현금흐름', '매출액']

In [42]:
value_index = data_fs.loc[value_type].iloc[:,[(len(data_fs.columns) - 1)]]
print(value_index)

                2019/12
IFRS(연결)               
지배주주순이익        215051.0
자본            2628804.0
영업활동으로인한현금흐름   453829.0
매출액           2304009.0


- 현재 주가 크롤링하기

In [43]:
url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A005930'

data = pd.read_html(url)[0]

In [44]:
data

,0,1,2,3
0,종가/ 전일대비,"83,500/ +1,000",거래량,18036835
1,52주.최고가/ 최저가,"91,000/ 42,500",거래대금(억원),14992
2,수익률(1M/ 3M/ 6M/ 1Y),-0.48/ +38.47/ +46.75/ +40.34,외국인 보유비중,54.93
3,"시가총액(상장예정포함,억원)",5596173,베타(1년),1.00725
4,"시가총액(보통주,억원)",4984768,액면가,100
5,NaN,NaN,NaN,NaN
6,발행주식수(보통주/ 우선주),"5,969,782,550/ 822,886,700",유동주식수/비율(보통주),"4,705,523,798 / 78.82"


In [45]:
price = data.iloc[0, 1].split()[0]
price = price.replace(",", "")
price = int(re.findall("[0-9]+", price)[0])

In [46]:
price

83500

In [47]:
share = data.iloc[6, 1].split()[0]
share = share.replace(",", "")
share = int(re.findall("[0-9]+", share)[0])

In [48]:
share

5969782550

In [66]:
import numpy as np

data_value = price / (value_index * 100000000 / share)
data_value.index = ['PER', 'PBR', 'PCR', 'PSR']
data_value[data_value.iloc[:, 0] < 0] = np.nan

In [67]:
data_value

,2019/12
PER,8.159266
PBR,0.812227
PCR,13.248969
PSR,0.804929


In [50]:
data_value.to_csv('data/KOR_value/005930_value.csv')

## 6.2.3 전 종목 재무제표 및 가치지표 다운로드

In [68]:
import pandas as pd
import numpy as np
import os
import time
import re

if not os.path.exists('data/KOR_fs'):
    os.makedirs('data/KOR_fs')
    
if not os.path.exists('data/KOR_value'):
    os.makedirs('data/KOR_value')

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)
error_ticker = []

for i in range(0, len(KOR_ticker)):
    
    # 빈 데이터프레임 생성
    data_fs = pd.DataFrame({'' : [np.nan]})
    data_value = pd.DataFrame({'' : [np.nan]})
    
    # 티커 선택
    name = KOR_ticker['종목코드'][i]
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        # url 생성
        url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A'+name
        data = pd.read_html(url)
        
        # 데이터 다운로드 후 테이블 추출
        data = pd.read_html(url)
                
        data_IS = data[0]
        data_BS = data[2]
        data_CF = data[4]
        data_IS = data_IS.iloc[:, 0:(len(data_IS.columns)-2)]
        
        # 클렌징
        data_fs = pd.concat([data_IS, data_BS, data_CF]).reset_index(drop=True)
        data_fs.iloc[:, 0] = data_fs.iloc[:, 0].replace({'계산에 참여한 계정 펼치기':''}, regex = True)
        data_fs = data_fs.set_index(data_fs.columns[0])
        data_fs = data_fs[~data_fs.index.duplicated(keep='first')]
        data_fs = data_fs.filter(like = '12', axis = 1)
        
        # 가치 지표 선택
        value_type = ['지배주주순이익', '자본', '영업활동으로인한현금흐름', '매출액']            
        value_index = data_fs.loc[value_type].iloc[:,[(len(data_fs.columns) - 1)]]    
        
        # 현재 주가
        url_snap = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A'+name
        data_snap = pd.read_html(url_snap)[0]
        
        price = data_snap.iloc[0, 1].split()[0]
        price = price.replace(",", "")
        price = int(re.findall("[0-9]+", price)[0])
        
        # 상장 주식수
        share = data_snap.iloc[6, 1].split()[0]
        share = share.replace(",", "")
        share = int(re.findall("[0-9]+", share)[0])
        
        data_value = price / (value_index * 100000000 / share)
        data_value.index = ['PER', 'PBR', 'PCR', 'PSR']
        data_value[data_value.iloc[:, 0] < 0] = np.nan     
        
    except:
        # 오류 발생시 해당 종목명을 출력하고 다음 루프로 이동
        # count_ticker에 오류발생 종목 저장, 마감 후 원인 파악
        print('Error in ticker ' + name)
        error_ticker.append(name)
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장    
    data_fs.to_csv('data/KOR_fs/'+name+'_fs.csv')
    data_value.to_csv('data/KOR_value/'+name+'_value.csv')
    
    # 타임슬립 적용
    time.sleep(2)

Error in ticker 105560
Error in ticker 055550
Error in ticker 032830
Error in ticker 086790
Error in ticker 000810
Error in ticker 316140
Error in ticker 006800
Error in ticker 024110
Error in ticker 071050
Error in ticker 016360
Error in ticker 029780
Error in ticker 336260
Error in ticker 039490
Error in ticker 005940
Error in ticker 005830
Error in ticker 088350
Error in ticker 008560
Error in ticker 357780
Error in ticker 058470
Error in ticker 235980
Error in ticker 001450
Error in ticker 138930
Error in ticker 000060
Error in ticker 323990
Error in ticker 336370
Error in ticker 141080
Error in ticker 268600
Error in ticker 003520
Error in ticker 064760
Error in ticker 138040
Error in ticker 298380
Error in ticker 139130
Error in ticker 175330
Error in ticker 178920
Error in ticker 363280
Error in ticker 314130
Error in ticker 005690
Error in ticker 033270
Error in ticker 003690
Error in ticker 213420
Error in ticker 131970
Error in ticker 058820
Error in ticker 001880
Error in ti

Error in ticker 054180
Error in ticker 069730
Error in ticker 274090
Error in ticker 020180
Error in ticker 110020
Error in ticker 277070
Error in ticker 097800
Error in ticker 126640
Error in ticker 147760
Error in ticker 008870
Error in ticker 169330
Error in ticker 066410
Error in ticker 115500
Error in ticker 069460
Error in ticker 025820
Error in ticker 318000
Error in ticker 039670
Error in ticker 019590
Error in ticker 228340
Error in ticker 105550
Error in ticker 097870
Error in ticker 065570
Error in ticker 017480
Error in ticker 062970
Error in ticker 036480
Error in ticker 017510
Error in ticker 024850
Error in ticker 021040
Error in ticker 317870
Error in ticker 111820
Error in ticker 189690
Error in ticker 039610
Error in ticker 051630
Error in ticker 154030
Error in ticker 147830
Error in ticker 087600
Error in ticker 059120
Error in ticker 128660
Error in ticker 010280
Error in ticker 010660
Error in ticker 220180
Error in ticker 119500
Error in ticker 050860
Error in ti

- 금융주의 경우 매출액이 없어 PSR 계산 시 에러 발생
- 지배주주순이익 없는 종목 에러 발생
- 12월 결산 아닌 종목 에러 발생

## 6.3 DART의 Open API를 이용한 데이터 수집하기

### 6.3.1 API Key발급 및 추가하기

In [ ]:
# import keyring
# keyring.set_password('Henry', 'dart_api_key', 'Your API Key')

In [1]:
import keyring
dart_api = keyring.get_password('Henry', 'dart_api_key')

### 6.3.2 고유번호 다운로드

In [2]:
codezip_url =  'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='+dart_api

In [3]:
import requests as rq
from io import BytesIO
import zipfile

codezip_data = rq.get(codezip_url)

In [4]:
codezip_data.headers['Content-Disposition']

': attachment; filename=CORPCODE.zip'

In [5]:
codezip_file = zipfile.ZipFile(BytesIO(codezip_data.content))

In [6]:
codezip_file.namelist()

['CORPCODE.xml']

In [7]:
code_data  = codezip_file.read('CORPCODE.xml').decode('utf-8')

In [8]:
import xmltodict
import json
import pandas as pd

data_odict = xmltodict.parse(code_data)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result').get('list')
corp_list = pd.DataFrame(data)

In [9]:
corp_list.head()

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630


In [16]:
corp_list = corp_list[~corp_list.stock_code.isin([None])].reset_index(drop=True)

In [17]:
corp_list.to_csv('data/corp_list.csv')

### 6.3.3 공시검색

#### 6.3.3.1 전체 공시 검색 공시 검색

In [18]:
from datetime import date
from dateutil.relativedelta import relativedelta

bgn_date = (date.today() + relativedelta(days=-7)).strftime("%Y-%m-%d").replace("-", "")
end_date = (date.today()).strftime("%Y-%m-%d").replace("-", "")

In [19]:
notice_url = 'https://opendart.fss.or.kr/api/list.json?crtfc_key='+dart_api+'&bgn_de='+bgn_date+'&end_de='+end_date+'&page_no=1&page_count=100'

In [20]:
notice_data  = rq.get(notice_url)
notice_data_df = notice_data.json().get('list')
notice_data_df = pd.DataFrame(notice_data_df)

In [21]:
notice_data_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00595012,애큐온캐피탈,,E,투자설명서(일괄신고),20210208000605,애큐온캐피탈,20210208,
1,00108940,대성홀딩스,016710,Y,증권신고서(채무증권),20210208000604,대성홀딩스,20210208,
2,00115694,DB금융투자,016610,Y,일괄신고추가서류(파생결합증권-주가연계증권),20210208000603,DB금융투자,20210208,
3,00140177,GS리테일,007070,Y,현금ㆍ현물배당결정,20210208800895,GS리테일,20210208,유
4,00585219,세원셀론텍,091090,Y,사외이사의선임ㆍ해임또는중도퇴임에관한신고,20210208000602,세원셀론텍,20210208,
...,...,...,...,...,...,...,...,...,...
95,00361008,HSD엔진,082740,Y,단기차입금증가결정,20210208800796,HSD엔진,20210208,유
96,00992543,삼양옵틱스,225190,K,현금ㆍ현물배당결정,20210208900770,삼양옵틱스,20210208,코
97,00300548,현대리바트,079430,Y,현금ㆍ현물배당결정,20210208800794,현대리바트,20210208,유
98,00372882,KTcs,058850,Y,연결재무제표기준영업(잠정)실적(공정공시),20210208800708,KTcs,20210208,유


#### 6.3.3.2 특정 기업의 공시 검색

In [22]:
corp_list[corp_list['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,modify_date
3083,00126380,삼성전자,005930,20201209


In [23]:
bgn_date = (date.today() + relativedelta(days=-30)).strftime("%Y-%m-%d").replace("-", "")
end_date = (date.today()).strftime("%Y-%m-%d").replace("-", "")
corp_code  = '00126380'

notice_url_ss =  'https://opendart.fss.or.kr/api/list.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bgn_de='+'&end_de='+'&page_no=1&page_count=100'

In [24]:
notice_data_ss  = rq.get(notice_url_ss)
notice_data_ss_df = notice_data_ss.json().get('list')
notice_data_ss_df = pd.DataFrame(notice_data_ss_df)

In [25]:
notice_data_ss_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210208000166,장세연,20210208,
1,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210208000100,조필주,20210208,
2,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210208000022,최호규,20210208,


In [26]:
notice_url_exam  = notice_data_ss_df.loc[0, 'rcept_no']
notice_dart_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo='+notice_url_exam

print(notice_dart_url)

http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210208000166


### 6.3.4 사업보고서 주요 정보

In [30]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'

url_div = 'https://opendart.fss.or.kr/api/alotMatter.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bsns_year='+bsns_year+'&reprt_code='+reprt_code

div_data_ss = rq.get(url_div)
div_data_ss_df = div_data_ss.json().get('list')
div_data_ss_df = pd.DataFrame(div_data_ss_df)

In [31]:
div_data_ss_df

,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr,stock_knd
0,20200330003851,Y,00126380,삼성전자,주당액면가액(원),100,100,100,NaN
1,20200330003851,Y,00126380,삼성전자,(연결)당기순이익(백만원),"21,505,054","43,890,877","41,344,569",NaN
2,20200330003851,Y,00126380,삼성전자,(별도)당기순이익(백만원),"15,353,323","32,815,127","28,800,837",NaN
3,20200330003851,Y,00126380,삼성전자,(연결)주당순이익(원),"3,166","6,461","5,997",NaN
4,20200330003851,Y,00126380,삼성전자,현금배당금총액(백만원),"9,619,243","9,619,243","5,826,302",NaN
5,20200330003851,Y,00126380,삼성전자,주식배당금총액(백만원),-,-,-,NaN
6,20200330003851,Y,00126380,삼성전자,(연결)현금배당성향(%),44.70,21.90,14.10,NaN
7,20200330003851,Y,00126380,삼성전자,현금배당수익률(%),2.60,3.70,1.70,보통주
8,20200330003851,Y,00126380,삼성전자,현금배당수익률(%),3.10,4.50,2.10,우선주
9,20200330003851,Y,00126380,삼성전자,주식배당수익률(%),-,-,-,보통주


### 6.3.5 상장기업 재무정보

#### 6.3.5.1 단일회사 및 다중회사 주요계정

In [33]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'

url_single = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bsns_year='+bsns_year+'&reprt_code='+reprt_code

In [34]:
fs_data_ss  = rq.get(url_single)
fs_data_ss_df = fs_data_ss.json().get('list')
fs_data_ss_df = pd.DataFrame(fs_data_ss_df)

In [39]:
fs_data_ss_df.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 51 기,2019.12.31 현재,"181,385,260,000,000",제 50 기,2018.12.31 현재,"174,697,424,000,000",제 49 기,2017.12.31 현재,"146,982,464,000,000",1
1,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 51 기,2019.12.31 현재,"171,179,237,000,000",제 50 기,2018.12.31 현재,"164,659,820,000,000",제 49 기,2017.12.31 현재,"154,769,626,000,000",3
2,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 51 기,2019.12.31 현재,"352,564,497,000,000",제 50 기,2018.12.31 현재,"339,357,244,000,000",제 49 기,2017.12.31 현재,"301,752,090,000,000",5
3,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 51 기,2019.12.31 현재,"63,782,764,000,000",제 50 기,2018.12.31 현재,"69,081,510,000,000",제 49 기,2017.12.31 현재,"67,175,114,000,000",7
4,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 51 기,2019.12.31 현재,"25,901,312,000,000",제 50 기,2018.12.31 현재,"22,522,557,000,000",제 49 기,2017.12.31 현재,"20,085,548,000,000",9


In [37]:
corp_code = ('00126380,00413046,00190321')
bsns_year = '2019'
reprt_code = '11011'

url_multiple = 'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bsns_year='+bsns_year+'&reprt_code='+reprt_code

In [40]:
fs_data_multiple  = rq.get(url_multiple)
fs_data_multiple_df = fs_data_multiple.json().get('list')
fs_data_multiple_df = pd.DataFrame(fs_data_multiple_df)

In [42]:
fs_data_multiple_df.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 51 기,2019.12.31 현재,"181,385,260,000,000",제 50 기,2018.12.31 현재,"174,697,424,000,000",제 49 기,2017.12.31 현재,"146,982,464,000,000",1
1,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 51 기,2019.12.31 현재,"171,179,237,000,000",제 50 기,2018.12.31 현재,"164,659,820,000,000",제 49 기,2017.12.31 현재,"154,769,626,000,000",3
2,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 51 기,2019.12.31 현재,"352,564,497,000,000",제 50 기,2018.12.31 현재,"339,357,244,000,000",제 49 기,2017.12.31 현재,"301,752,090,000,000",5
3,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 51 기,2019.12.31 현재,"63,782,764,000,000",제 50 기,2018.12.31 현재,"69,081,510,000,000",제 49 기,2017.12.31 현재,"67,175,114,000,000",7
4,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 51 기,2019.12.31 현재,"25,901,312,000,000",제 50 기,2018.12.31 현재,"22,522,557,000,000",제 49 기,2017.12.31 현재,"20,085,548,000,000",9


In [47]:
fs_data_list = dict(tuple(fs_data_multiple_df.groupby('corp_code')))

In [52]:
fs_data_list['00126380'].head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 51 기,2019.12.31 현재,"181,385,260,000,000",제 50 기,2018.12.31 현재,"174,697,424,000,000",제 49 기,2017.12.31 현재,"146,982,464,000,000",1
1,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 51 기,2019.12.31 현재,"171,179,237,000,000",제 50 기,2018.12.31 현재,"164,659,820,000,000",제 49 기,2017.12.31 현재,"154,769,626,000,000",3
2,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 51 기,2019.12.31 현재,"352,564,497,000,000",제 50 기,2018.12.31 현재,"339,357,244,000,000",제 49 기,2017.12.31 현재,"301,752,090,000,000",5
3,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 51 기,2019.12.31 현재,"63,782,764,000,000",제 50 기,2018.12.31 현재,"69,081,510,000,000",제 49 기,2017.12.31 현재,"67,175,114,000,000",7
4,20200330003851,11011,2019,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 51 기,2019.12.31 현재,"25,901,312,000,000",제 50 기,2018.12.31 현재,"22,522,557,000,000",제 49 기,2017.12.31 현재,"20,085,548,000,000",9


In [53]:
fs_data_list['00413046'].head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
52,20200410002837,11011,2019,00413046,068270,CFS,연결재무제표,BS,재무상태표,유동자산,제 29 기,2019.12.31 현재,"1,787,340,254,600",제 28 기,2018.12.31 현재,"1,664,478,918,682",제 27 기,2017.12.31 현재,"1,614,033,788,024",1
53,20200410002837,11011,2019,00413046,068270,CFS,연결재무제표,BS,재무상태표,비유동자산,제 29 기,2019.12.31 현재,"2,106,351,351,846",제 28 기,2018.12.31 현재,"1,876,147,755,272",제 27 기,2017.12.31 현재,"1,701,493,916,629",3
54,20200410002837,11011,2019,00413046,068270,CFS,연결재무제표,BS,재무상태표,자산총계,제 29 기,2019.12.31 현재,"3,893,691,606,446",제 28 기,2018.12.31 현재,"3,540,626,673,954",제 27 기,2017.12.31 현재,"3,315,527,704,653",5
55,20200410002837,11011,2019,00413046,068270,CFS,연결재무제표,BS,재무상태표,유동부채,제 29 기,2019.12.31 현재,"657,225,837,904",제 28 기,2018.12.31 현재,"678,087,471,395",제 27 기,2017.12.31 현재,"616,781,029,714",7
56,20200410002837,11011,2019,00413046,068270,CFS,연결재무제표,BS,재무상태표,비유동부채,제 29 기,2019.12.31 현재,"329,507,580,221",제 28 기,2018.12.31 현재,"229,761,340,951",제 27 기,2017.12.31 현재,"266,817,280,825",9


In [54]:
fs_data_list['00190321'].head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
26,20200330004658,11011,2019,00190321,030200,CFS,연결재무제표,BS,재무상태표,유동자산,제 38 기,2019.12.31 현재,"11,898,255,000,000",제 37 기,2018.12.31 현재,"11,894,252,000,000",제 36 기,2017.12.31 현재,"9,672,412,000,000",1
27,20200330004658,11011,2019,00190321,030200,CFS,연결재무제표,BS,재무상태표,비유동자산,제 38 기,2019.12.31 현재,"22,163,037,000,000",제 37 기,2018.12.31 현재,"20,294,578,000,000",제 36 기,2017.12.31 현재,"20,058,498,000,000",3
28,20200330004658,11011,2019,00190321,030200,CFS,연결재무제표,BS,재무상태표,자산총계,제 38 기,2019.12.31 현재,"34,061,292,000,000",제 37 기,2018.12.31 현재,"32,188,830,000,000",제 36 기,2017.12.31 현재,"29,730,910,000,000",5
29,20200330004658,11011,2019,00190321,030200,CFS,연결재무제표,BS,재무상태표,유동부채,제 38 기,2019.12.31 현재,"10,111,456,000,000",제 37 기,2018.12.31 현재,"9,387,704,000,000",제 36 기,2017.12.31 현재,"9,474,162,000,000",7
30,20200330004658,11011,2019,00190321,030200,CFS,연결재무제표,BS,재무상태표,비유동부채,제 38 기,2019.12.31 현재,"8,762,883,000,000",제 37 기,2018.12.31 현재,"8,069,846,000,000",제 36 기,2017.12.31 현재,"7,046,148,000,000",9


### 6.3.6 단일회사 전체 재무제표

In [61]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'

url_fs_all = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bsns_year='+bsns_year+'&reprt_code='+reprt_code+'&fs_div=CFS'

In [62]:
url_fs_all

'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=b1a630e527b0e5ff5bd58ed81b49825017fa80b8&corp_code=00126380&bsns_year=2019&reprt_code=11011&fs_div=CFS'

In [63]:
fs_data_all  = rq.get(url_fs_all)
fs_data_all_df = fs_data_all.json().get('list')
fs_data_all_df = pd.DataFrame(fs_data_all_df)

In [64]:
fs_data_all_df

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
0,20200330003851,11011,2019,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 51 기,181385260000000,제 50 기,174697424000000,제 49 기,146982464000000,1,NaN
1,20200330003851,11011,2019,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 51 기,26885999000000,제 50 기,30340505000000,제 49 기,30545130000000,2,NaN
2,20200330003851,11011,2019,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 51 기,76252052000000,제 50 기,65893797000000,제 49 기,49447696000000,3,NaN
3,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,단기매도가능금융자산,-,제 51 기,,제 50 기,,제 49 기,3191375000000,4,NaN
4,20200330003851,11011,2019,00126380,BS,재무상태표,-표준계정코드 미사용-,단기상각후원가금융자산,-,제 51 기,3914216000000,제 50 기,2703693000000,제 49 기,,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,20200330003851,11011,2019,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분,제 51 기,254915472000000,제 50 기,240068993000000,제 49 기,207213416000000,17,NaN
206,20200330003851,11011,2019,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|기타자본항목,제 51 기,-4968829000000,제 50 기,-7931370000000,제 49 기,-13899191000000,17,NaN
207,20200330003851,11011,2019,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|이익잉여금 [member],제 51 기,254582894000000,제 50 기,242698956000000,제 49 기,215811200000000,17,NaN
208,20200330003851,11011,2019,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|자본금 [member],제 51 기,897514000000,제 50 기,897514000000,제 49 기,897514000000,17,NaN
